In [1]:
import seabird
import datetime
import numpy as np
import seapy
import gsw
import scipy as sp

In [67]:
from seabird.cnv import fCNV
ctd_file = '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012AUTUMN/CTD/3600-01acfld.cnv'
profile = fCNV(ctd_file)


In [5]:
print("Header: %s" % profile.attributes.keys())
print(profile.attributes)


Header: dict_keys(['sbe_model', 'seasave', 'instrument_type', 'nquan', 'nvalues', 'start_time', 'bad_flag', 'file_type', 'md5', 'datetime', 'filename'])
{'sbe_model': '9', 'seasave': 'Win32 V 5.39c', 'instrument_type': 'CTD', 'nquan': '19', 'nvalues': '8272', 'start_time': 'Dec 13 2012 09:13:57 [System UpLoad Time]', 'bad_flag': '-9.990e-29', 'file_type': 'ascii', 'md5': '7a21b5b1b1bf45d261940c23b8a6c88f', 'datetime': datetime.datetime(2012, 12, 13, 9, 13, 57), 'filename': '3600-01acfld.cnv'}


In [6]:
print(profile.keys())

['PRES', 'TEMP', 'CNDC', 'pumps', 'PSAL', 'altM', 'descentrate', 'accM', 'scan', 'timeS', 'flSP', 'seaTurbMtr', 'potemperature', 'PSAL', 'density', 'sigma_t', 'soundspeed', 'DEPTH', 'flag']


In [95]:
invalid =np.ma.getmaskarray(profile['flag'])

In [98]:
invalid

array([False, False, False, ...,  True,  True,  True])

In [100]:
profile['timeS'][~invalid].shape

(2146,)

In [102]:
#PSAL
profile.data[-6]

masked_array(data=[2.979, 2.9559, 2.9285, ..., 6.2836, 6.2783, --],
             mask=[False, False, False, ..., False, False,  True],
       fill_value=-9.99e-29)

In [37]:
profile.attrs['LATITUDE']=36.0045 
profile.attrs['LONGITUDE']=123.9935

In [42]:
profile.attrs.datetime

AttributeError: 'dict' object has no attribute 'datetime'

In [41]:
df_profile

{}

In [39]:
def lanczos_filter(data,Cf,Nf,M):
    coef = sp.signal.firwin(M+1, Cf/Nf,width = 2/len(data),window='lanczos', pass_zero='lowpass')
    return sp.signal.filtfilt(coef,1.0,data)

dt = 0.04
fs = 1/(dt)
Nf = fs/2
Cf = fs/3000
M = 100

# fdiff_pres = lanczos_filter(diff_pres,Cf,Nf,M)

In [86]:
diff_pres

masked_array(data=[0.03735033027343726, 0.03741818638857275,
                   0.03748076236520781, ..., --, --, --],
             mask=[False, False, False, ...,  True,  True,  True],
       fill_value=1e+20)

In [83]:
%matplotlib
import matplotlib.pyplot as plt

time = profile['timeS'][:]
time[invalid] = np.ma.masked
# depth = profile['DEPTH'][:]
temp = profile['TEMP'][:]
pres = profile['PRES'][:]

temp[invalid] = np.ma.masked
pres[invalid] = np.ma.masked
plt.plot(time, pres,'b')

pres_conv = np.convolve(pres,np.ones(24*3)/24/3,mode='full')[36:-35]
# pres_conv = lanczos_filter(pres,Cf,Nf,M)
# pres_conv = seapy.filt.bandpass(pres,dt,low_cutoff=dt*300,order=7)
pres_conv = seapy.filt.bandpass(pres_conv,dt,low_cutoff=dt*300,order=7)
pres_conv[invalid] = np.ma.masked
# plt.plot(time,pres_conv,'k--')
diff_pres = np.diff(pres_conv)
diff_pres_conv = diff_pres.copy()
# plt.plot(time[:-1],diff_pres)
plt.xlabel('TIME(sec)')
plt.ylabel('PRES(db)')
plt.title('H01')
# diff_pres_conv = np.convolve(diff_pres,np.ones(24*3)/24/3,mode='full')[36:-35]
# diff_pres_conv = lanczos_filter(diff_pres,Cf,Nf,M)
# diff_pres_conv = seapy.filt.bandpass(diff_pres,dt,low_cutoff=dt*600)
# plt.plot(time[:-1],diff_pres_conv)



# pres_conv = lanczos_filter(pres,Cf,Nf,M)
# diff_pres_conv = np.diff(pres_conv)


Using matplotlib backend: MacOSX


Text(0.5, 1.0, 'H01')

In [7]:
profile['PRES'][:]

masked_array(data=[0.065, 0.062, 0.058, ..., -0.298, -0.298, -0.298],
             mask=[False, False, False, ..., False, False, False],
       fill_value=-9.99e-29)

In [8]:

pres = np.asarray([1,2,3,4,5,4,3,2,1,3,4,5,6,4,3,2])
print(len(pres))
diff_pres = np.diff(pres)
print(diff_pres)

16
[ 1  1  1  1 -1 -1 -1 -1  2  1  1  1 -2 -1 -1]


In [29]:
np.diff(time)

masked_array(data=[0.042, 0.041, 0.042, ..., 0.042, 0.041, 0.042],
             mask=False,
       fill_value=-9.99e-29)

In [93]:
len(diff_pres_conv)

8271

In [94]:
count_down = 0
count_up = 0
for i in range(1,len(diff_pres_conv)):
    # print(diff_pres[i])
    # print(diff_pres[i]*diff_pres[i-1])
    if (diff_pres_conv.data[i]*diff_pres_conv.data[i-1]) < 0 :
        if diff_pres_conv.data[i-1]>0:
            if  not count_down: 
                start_down = i
            else:
                start_down = np.r_[start_down, i]
            count_down+=1
        else:
            if not count_up : 
                start_up = i
            else:
                start_up = np.r_[start_up, i]
            count_up+=1
            

# print(start_up)          
# print(start_down)


# print(pres[start_down[1]])
larger = 1
smaller = -1
equal = 0
print(len(start_up))
print(len(start_down))

if len(start_up) > len(start_down):
    end_indx = len(start_down)
    status = larger
elif len(start_up) < len(start_down):
    end_indx = len(start_up)
    status = smaller
else:
    
    status = equal
    


if start_down[0] < start_up[0]:
    if status == equal:
        mono_duration = start_up[:-1] - start_down[1:]
    elif status == larger:
        mono_duration = start_up[:end_indx-1] - start_down[1:]
    elif status == smaller:
        mono_duration = start_up[:] - start_down[1:1+len(start_up)]
    # print(mono_duration)
    downcast_index = np.argmax(np.abs(mono_duration))
    print(downcast_index)
    if ((pres[start_down[downcast_index+1]] - 
              pres[start_up[downcast_index]]) < 5):
        mono_duration[downcast_index] = 0
        downcast_index = np.argmax(np.abs(mono_duration))
    mono_duration[downcast_index] = 0
    downcast2_index = np.argmax(np.abs(mono_duration))
    if (downcast2_index > downcast_index)  \
        and ((pres[start_down[downcast2_index+1]] - 
              pres[start_up[downcast2_index]]) > 5):
        time_downcast = np.r_[
                              time[start_up[downcast_index]:start_down[downcast_index+1]],
                              time[start_up[downcast2_index]:start_down[downcast2_index+1]]

                              ]
        pres_downcast = np.r_[
                              pres[start_up[downcast_index]:start_down[downcast_index+1]],
                              pres[start_up[downcast2_index]:start_down[downcast2_index+1]]
                              ]
        temp_downcast = np.r_[
                               temp[start_up[downcast_index]:start_down[downcast_index+1]],
                               temp[start_up[downcast2_index]:start_down[downcast2_index+1]]
                            ]
    else:
            
        time_downcast = time[start_up[downcast_index]:start_down[downcast_index+1]]
        pres_downcast = pres[start_up[downcast_index]:start_down[downcast_index+1]]
        temp_downcast = temp[start_up[downcast_index]:start_down[downcast_index+1]]

else:
    if status == equal:
        mono_duration = start_up - start_down
    elif status == larger:
        mono_duration = start_up[:end_indx] - start_down
    elif status == smaller:
        mono_duration = start_up - start_down[:end_indx]
    
    # print(mono_duration)
    downcast_index = np.argmax(np.abs(mono_duration))
    print(downcast_index)
    if ((pres[start_down[downcast_index]] - 
              pres[start_up[downcast_index]]) < 5):
        mono_duration[downcast_index] = 0
        downcast_index = np.argmax(np.abs(mono_duration))
    mono_duration[downcast_index] = 0
    downcast2_index = np.argmax(np.abs(mono_duration))
    print(downcast2_index)
    if (downcast2_index > downcast_index)\
        and ((pres[start_down[downcast2_index]] - 
              pres[start_up[downcast2_index]]) > 5):
        time_downcast = np.r_[
                              time[start_up[downcast_index]:start_down[downcast_index]],
                              time[start_up[downcast2_index]:start_down[downcast2_index]]

                              ]
        pres_downcast = np.r_[
                              pres[start_up[downcast_index]:start_down[downcast_index]],
                              pres[start_up[downcast2_index]:start_down[downcast2_index]]
                              ]
        temp_downcast = np.r_[
                               temp[start_up[downcast_index]:start_down[downcast_index]],
                               temp[start_up[downcast2_index]:start_down[downcast2_index]]
                            ]
    else:
            
        time_downcast = time[start_up[downcast_index]:start_down[downcast_index]]
        pres_downcast = pres[start_up[downcast_index]:start_down[downcast_index]]
        temp_downcast = temp[start_up[downcast_index]:start_down[downcast_index]]

# print(status)
print(time[start_up[downcast_index]])
print(time[start_down[downcast_index+1]])
# print(np.argmax(np.abs(start_down-start_up)))
# print(start_up[downcast_index])

14
15
2
--
--


In [70]:

time_downcast = time[start_up[downcast_index]:start_down[downcast_index+1]]
pres_downcast = pres[start_up[downcast_index]:start_down[downcast_index+1]]
temp_downcast = temp[start_up[downcast_index]:start_down[downcast_index+1]]

In [71]:
time_downcast

masked_array(data=[235.833, 235.875, 235.917, 235.958, 236.0, 236.042,
                   236.083, 236.125, 236.167, 236.208, 236.25, 236.292,
                   236.333, 236.375, 236.417, 236.458, 236.5, 236.542,
                   236.583, 236.625, 236.667, 236.708, 236.75, 236.792,
                   236.833, 236.875, 236.917, 236.958, 237.0, 237.042,
                   237.083, 237.125, 237.167, 237.208, 237.25, 237.292,
                   237.333, 237.375, 237.417, 237.458, 237.5, 237.542,
                   237.583, 237.625, 237.667, 237.708, 237.75, 237.792,
                   237.833, 237.875, 237.917, 237.958, 238.0, 238.042,
                   238.083, 238.125, 238.167, 238.208, 238.25, 238.292,
                   238.333, 238.375, 238.417, 238.458, 238.5, 238.542,
                   238.583, 238.625, 238.667, 238.708, 238.75, 238.792,
                   238.833, 238.875, 238.917, 238.958, 239.0, 239.042,
                   239.083, 239.125, 239.167, 239.208, 239.25, 239.292,

In [51]:
Cf

0.0025

In [162]:
%matplotlib
freq, ps, psd,conf = seapy.filt.powerspectra(diff_pres)
# ffreq, fps, fpsd,fconf = seapy.filt.powerspectra(fdiff_pres)
# plt.loglog(np.full_like(freq,1/12,), np.linspace(0,10,len(freq)),'k')
plt.loglog(freq,psd*freq,label = 'raw')
# plt.loglog(ffreq,fpsd*ffreq,label = 'filted')

Using matplotlib backend: MacOSX


In [24]:
%matplotlib
plt.plot(time, pres,'b')
# plt.plot(time,fpres,'r--')
plt.plot(time,pres_conv,'k--')
# plt.plot(time,pres_conv,'k--')
# plt.plot(time[start_up[downcast2_index]:start_down[downcast2_index+1]],pres[start_up[downcast2_index]:start_down[downcast2_index+1]],'r')
plt.plot(time_downcast,pres_downcast,'r')
# plt.plot(time[:-1],diff_pres)
plt.plot(time[start_up],pres[start_up],'r.',label='start_up')
plt.plot(time[start_down],pres[start_down],'g.',label='start_down')
plt.xlabel('TIME(sec)')
plt.ylabel('PRES(db)')
plt.legend()
plt.title('E01')

Using matplotlib backend: MacOSX


Text(0.5, 1.0, 'E01')

In [26]:
%matplotlib
plt.plot(temp[start_up[downcast2_index]:start_down[downcast2_index+1]], pres[start_up[downcast2_index]:start_down[downcast2_index+1]],'b')
plt.plot(temp_downcast, pres_downcast,'b')
plt.gca().invert_yaxis()
plt.xlabel('TEMP')
plt.ylabel('PRES')
# plt.xlim([5.5,17])
plt.title('Downcast_Software')

Using matplotlib backend: MacOSX


Text(0.5, 1.0, 'Downcast_Software')

In [27]:
%matplotlib
plt.plot(temp[start_up[downcast_index]:start_down[downcast_index+1]], pres[start_up[downcast_index]:start_down[downcast_index+1]],'b')
plt.gca().invert_yaxis()
plt.xlabel('TEMP')
plt.ylabel('PRES')
# plt.xlim([5.5,17])
plt.title('Downcast')

Using matplotlib backend: MacOSX


Text(0.5, 1.0, 'Downcast')

In [152]:
%matplotlib
plt.plot(temp,pres,'b')
plt.gca().invert_yaxis()
plt.xlabel('TEMP')
plt.ylabel('PRES')
plt.xlim([5.5,17])
plt.title('H04 Bin_Averaged')

Using matplotlib backend: MacOSX


(5.5, 17.0)

In [ ]:
%matplotlib
plt.plot(np.convolve(temp_downcast,np.ones(48)/48,mode='full')[24:-23],np.convolve(pres_downcast,np.ones(48)/48,mode='full')[24:-23],'b')
plt.gca().invert_yaxis()
plt.xlabel('TEMP')
plt.ylabel('PRES')
plt.title('Bin_Averaged')

In [40]:
np.ceil(np.max(pres))

32.0

In [43]:
# time_valid = np.ma.masked_outside(time,124.0,190)
# depth[time_valid.mask] = np.ma.masked
# temp[time_valid.mask] = np.ma.masked
# pres[time_valid.mask] = np.ma.masked


# z = -gsw.z_from_p(pres, 35.9648)
# z_intp = np.arange(3,39,3)
pres_intp = np.arange(1, np.ceil(np.max(pres_downcast)))
temp_intp = np.interp(pres_intp,pres_downcast,temp_downcast)

In [44]:
%matplotlib
import matplotlib.pyplot as plt
# plt.plot(temp_downcast,pres_downcast,'b')
plt.plot(temp_intp, pres_intp,'b')
# plt.plot(temp_raw, pres_intp,'k')
# plt.plot(temp_raw-temp_intp, z_intp,'k')
plt.xlim([5.5,17])
plt.gca().invert_yaxis()
plt.xlabel('TEMP')
plt.ylabel('PRES')
plt.title('H04 Bin_Averaged')

Using matplotlib backend: MacOSX


Text(0.5, 1.0, 'H04 Bin_Averaged')

In [ ]:
temp_raw = np.asarray([14.7705,13.9148,9.3748,9.3309,9.3286,9.3283,9.3413,9.3312,9.3314,9.3485])

temp_raw = np.asarray([14.8674,13.1768,9.8430,9.1345,8.0230,7.9294,7.9236,7.9091,7.9076,7.9169,7.9143,7.9101])

In [ ]:
temp_raw.shape

In [ ]:
temp_intp.shape